# Introduction

This notebook presents a benchmark comparison between `power-grid-model` and 
[`pandapower`](https://github.com/e2nIEE/pandapower).
It runs several calculations, measures the calculation time, and compares the results.

## Test Network

The test network is fictionally generated using pre-defined random criteria. It is a radial network as follows:

```

source --- source_node ---| ---line--- node ---line--- node ...   (n_node_per_feeder)
                          |              |              |
                          |            load            load ...
                          |
                          | ---line--- ...
                          | .
                          | .
                          | .
                          | (n_feeder)

```

There is a node which is connected to a source (external network). From the source node there are `n_feeder` feeders. For each feeder there are `n_node_per_feeder` nodes, lines, and asymmetric loads. There are in total `n_feeder * n_node_per_feeder + 1` nodes in the network.


## Calculation

The notebook runs a power flow calculation with the same input data in `power-grid-model` and `pandapower`. It runs the following calculations:

* Single calculation with solver initialization.
* Single calculation without solver initialization (using pre-cached internal matrices).
* Time-series calculation
* N-1 calculation

The calculation is run symmetrically and asymmetrically. Both `power-grid-model` and `pandapower` supports asymmetric loads in symmetric calculations: the three-phase load is aggregated into one symmetric load.

We use the Newton-Raphson method for both libraries. In addition, we also use the linear method in `power-grid-model` to see how much performance you can gain in exchange for accuracy.

## Results Comparison

The following results are compared between `power-grid-model` and `pandapower`:

* Per unit voltage of nodes (buses). For asymmetric calculation, it compares the value per phase.
* Loading of the lines.

It only compares the results of the Newton-Raphson method, since the linear method of `power-grid-model` will produce a less accurate result.

## Performance Benchmark

The CPU time is measured for the **calculation part** of the program. The data preparation and model initialization is not measured. Furthermore, the single calculation is benchmarked with and without solver initialization. The former needs to execute connectivity check and initialize internal matrices (for example node admittance matrix). The latter uses the pre-cached connectivity and internal matrices.

# Preparation

## Import Libraries

We import neede libraries here. The fictional network generation and time-series profile generation is in a different Python file
[generate_fictional_dataset.py](./generate_fictional_dataset.py).

In [1]:
import tempfile
import time
from pathlib import Path

import numpy as np
import pandapower as pp
import pandas as pd
import power_grid_model as pgm

from generate_fictional_dataset import generate_fictional_grid, generate_time_series

# output dir for pandapower time-series
output_dir = Path(tempfile.gettempdir()) / "pandapower_time_series"
output_dir.mkdir(exist_ok=True)

## Prepare Tables

The performance comparison table and result deviation table is initialized below.

## Simulation Parameters

The simulation parameters, for example, the total number of feeders `n_feeder`, are defined below.

In [2]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = 1000
n_node = n_node_per_feeder * n_feeder + 1
n_line = n_node_per_feeder * n_feeder
n_load = n_node_per_feeder * n_feeder

use_lightsim2grid = True

# Single Calculation

We begin with a single power flow calculation. First generate the fictional datasets.

In [3]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pgm_dataset = fictional_dataset['pgm_dataset']

# Time Series Calculation

We execute a time-series power flow with `n_step` timestamps. 

## Preparation

The load profile is randomly generated by the function `generate_time_series`. It produces the relevant input format for both libraries.

In [4]:
time_series_dataset = generate_time_series(
    fictional_dataset,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max
)

pgm_update_dataset = time_series_dataset['pgm_update_dataset']
pp_dataset = time_series_dataset['pp_dataset']
time_steps = np.arange(n_step)


## Asymmetric

### Newton-Raphson Method of power-grid-model

In [5]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, update_data=pgm_update_dataset, threading=1)
end = time.time()
print(end - start)

3.5031449794769287


# N-1 Scenario Calculation

We execute a N-1 scenario calculation. There are `n_line` scenarios. In each scenario one line will be disabled. Since the original network is radial, part of the network will be unenergized due to the switch-off of a line.

## Preparation

The N-1 scenario input is generated for `power-grid-model`. Since `pandapower` does not have a built-in N-1 scenario calculation, we execute the power flow per scenario in a loop.

In [6]:
# re-generate dataset
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']

# update dataset for power grid model
# disable one line per batch
pgm_line_profile = pgm.initialize_array('update', 'line', (n_line, n_line))
pgm_line_profile['id'] = pgm_dataset['line']['id']
pgm_line_profile['from_status'] = 1
pgm_line_profile['to_status'] = 1
np.fill_diagonal(pgm_line_profile['from_status'], 0)
np.fill_diagonal(pgm_line_profile['to_status'], 0)
pgm_update_dataset = {'line': pgm_line_profile}




## Asymmetric

### Newton-Raphson Method of power-grid-model

In [7]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(update_data=pgm_update_dataset, symmetric=False, threading=1)
end = time.time()
print(end - start)

3.62314510345459
